In [285]:
import os

import pandas as pd

train_path = '../data/processed/lang_detect/'
os.makedirs(train_path, exist_ok=True)

df_kbd = pd.read_csv('../data/processed/lang_detect/source.kbd.glossary.csv')
df_ady = pd.read_csv('../data/processed/lang_detect/source.ady.glossary.csv')
df_rus = pd.read_csv('../data/processed/lang_detect/source.rus.glossary.csv')

df_kbd.rename(columns={'#name': 'key', 'Адыгэбзэ псалъалъэ (Ady) (eastern)': 'value'}, inplace=True)
df_kbd['lang'] = 'kbd'
df_kbd['key'] = df_kbd['key'].apply(lambda x: str(x).lower().replace('i', 'I'))

df_ady.rename(columns={'#name': 'key', 'Адыгабзэм изэхэф гущыIалъ (Ady) (western)': 'value'}, inplace=True)
df_ady['lang'] = 'ady'

df_rus.rename(columns={'#name': 'key', 'Урыс-адыгэ школ псалъалъэ (Rus-Ady) (eastern)': 'value'}, inplace=True)
df_rus['lang'] = 'rus'

In [378]:
df_ady['-1_sent'] = df_ady['value'].apply(lambda x: x.split('\n')[-1] + '.')
with open(os.path.join(train_path, 'sentences_1.ady.txt'), 'w') as f:
    f.write('\n'.join(sorted(set(df_ady['-1_sent'].tolist()))))

df_kbd['-1_sent'] = df_kbd['value'].apply(lambda x: x.split('\n')[-1])
with open(os.path.join(train_path, 'sentences_1.kbd.txt'), 'w') as f:
    f.write('\n'.join(sorted(set(df_kbd['-1_sent'].tolist()))))

IndexError: list index out of range

In [270]:
kbd_words = sorted(set(df_kbd['key'].to_list()))
with open(os.path.join(train_path, 'words.kbd.txt'), 'w') as f:
    f.write('\n'.join(kbd_words))

ady_words = sorted(set(df_ady['key'].to_list()))
with open(os.path.join(train_path, 'words.ady.txt'), 'w') as f:
    f.write('\n'.join(ady_words))

rus_words = sorted(set(df_rus['key'].to_list()))
with open(os.path.join(train_path, 'words.rus.txt'), 'w') as f:
    f.write('\n'.join(rus_words))


In [365]:
import nltk

from src.logic.text_cleaner import clean_text

with open(os.path.join(train_path, 'source.rus.txt')) as f:
    rus_text = clean_text(f.read())

with open(os.path.join(train_path, 'source.kbd.txt')) as f:
    kbd_text = clean_text(f.read())

with open(os.path.join(train_path, 'source.ady.txt')) as f:
    ady_text = clean_text(f.read())

# rus_sentences = [sent.replace('\n', ' ') for sent in nltk.sent_tokenize(rus_text) if 20 <= len(sent) <= 80]
# kbd_sentences = [sent.replace('\n', ' ') for sent in nltk.sent_tokenize(kbd_text) if 20 <= len(sent) <= 80]
# ady_sentences = [sent.replace('\n', ' ') for sent in nltk.sent_tokenize(ady_text) if 20 <= len(sent) <= 80]

# with open(os.path.join(train_path, 'sentences.rus.txt'), 'w') as f:
#     f.write('\n'.join(sorted(set(rus_sentences))))

# with open(os.path.join(train_path, 'sentences.kbd.txt'), 'w') as f:
#     f.write('\n'.join(sorted(set(kbd_sentences))))

# with open(os.path.join(train_path, 'sentences.ady.txt'), 'w') as f:
#     f.write('\n'.join(sorted(set(ady_sentences))))

In [366]:
with open(os.path.join(train_path, 'sentences.rus.txt'), 'r') as f:
    rus_sentences = f.read().split('\n')

with open(os.path.join(train_path, 'sentences.kbd.txt'), 'r') as f:
    kbd_sentences = f.read().split('\n')

with open(os.path.join(train_path, 'sentences.ady.txt'), 'r') as f:
    ady_sentences = f.read().split('\n')

rus_sentences = [sent for sent in rus_sentences if 20 <= len(sent) <= 80]
kbs_sentences = [sent for sent in kbd_sentences if 20 <= len(sent) <= 80]
kbd_sentences = [sent for sent in kbd_sentences if 20 <= len(sent) <= 80]


In [367]:
import random

for seq in [
    rus_sentences,
    rus_words,
    kbd_sentences,
    kbd_words,
    ady_sentences,
    ady_words
]:
    random.shuffle(rus_sentences)

In [374]:
sent_n = 2000

rus_words_with_label = [f'__label__rus {word}' for word in rus_words][:5000]
kbd_words_with_label = [f'__label__kbd {word}' for word in kbd_words][:5000]
ady_words_with_label = [f'__label__ady {word}' for word in ady_words]

rus_sentences_with_label = [f'__label__rus {sent}' for sent in rus_sentences[:int(sent_n * 1.0)]]
kbd_sentences_with_label = [f'__label__kbd {sent}' for sent in kbd_sentences[:int(sent_n * 3.0)]]
ady_sentences_with_label = [f'__label__ady {sent}' for sent in ady_sentences[:int(sent_n * 10.0)]]

sentence_with_label = (
        rus_sentences_with_label + rus_words_with_label +
        kbd_sentences_with_label + kbd_words_with_label +
        ady_sentences_with_label + ady_words_with_label
)
random.shuffle(sentence_with_label)

train_sent = sentence_with_label[:int(len(sentence_with_label) * 0.8)]
test_sent = sentence_with_label[int(len(sentence_with_label) * 0.8):]

train_file_path = os.path.join(train_path, 'data.train.txt')
test_file_path = os.path.join(train_path, 'data.test.txt')

with open(train_file_path, 'w') as f:
    f.write('\n'.join(train_sent))

with open(test_file_path, 'w') as f:
    f.write('\n'.join(test_sent))

In [375]:
import fasttext

model = fasttext.train_supervised(input=train_file_path, epoch=1000)

Read 0M words
Number of words:  74840
Number of labels: 3
Progress: 100.0% words/sec/thread: 1900999 lr:  0.000000 avg.loss:  0.023881 ETA:   0h 0m 0s


In [409]:
score = model.test_label(test_file_path)
score

{'__label__rus': {'precision': 0.9878958587088916,
  'recall': nan,
  'f1score': 1.9757917174177833},
 '__label__kbd': {'precision': 0.9274903687396808,
  'recall': nan,
  'f1score': 1.8549807374793617},
 '__label__ady': {'precision': 0.5155227969977039,
  'recall': nan,
  'f1score': 1.0310455939954077}}

In [381]:
rus_precision = score['__label__rus']['precision']
kbd_precision = score['__label__kbd']['precision']
ady_precision = score['__label__ady']['precision']

In [384]:
model.save_model(f'../data/processed/lang_detect/models/rus_{rus_precision:.2f}__kbd_{kbd_precision:.2f}__ady_{ady_precision:.2f}.model')